# Fetch Data From Ids

In [47]:
import requests
import pandas as pd
import os
import time
from tqdm import tqdm

In [48]:
COVALENT_API_KEY = "YOUR COVALENT_API_KEY"
CHAIN_ID = 1  # Ethereum mainnet

In [49]:
# Compound V2 contract addresses (optional: narrow filtering)
COMPOUND_V2_ADDRESSES = {
    "cETH": "0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5",
    "cDAI": "0x5d3a536e4d6dbd6114cc1ead35777bab948e3643",
    "cUSDC": "0x39aa39c021dfbae8fac545936693ac917d5e7563",
    # Add more tokens as needed
}

In [50]:
def fetch_transactions(wallet_address):
    url = f"https://api.covalenthq.com/v1/{CHAIN_ID}/address/{wallet_address}/transactions_v2/"
    params = {
        "key": COVALENT_API_KEY,
        "page-size": 1000
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed for {wallet_address} | Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception for {wallet_address}: {e}")
        return None

In [51]:
def save_wallet_data(wallet_address, data, output_dir="raw_data"):
    os.makedirs(output_dir, exist_ok=True)
    filepath = os.path.join(output_dir, f"{wallet_address}.json")
    with open(filepath, "w") as f:
        json.dump(data, f, indent=2)

In [52]:
def main():
    # Load wallets
    df = pd.read_csv("wallet_id.csv")
    wallets = df['wallet_id'].dropna().unique().tolist()

    print(f"📦 Fetching data for {len(wallets)} wallets...\n")

    for wallet in tqdm(wallets):
        data = fetch_transactions(wallet)
        if data:
            save_wallet_data(wallet, data)
        time.sleep(0.5)  # To avoid rate limits

    print("\n✅ Done. All raw transaction data saved in 'raw_data/' folder.")

In [53]:
if __name__ == "__main__":
    main()

📦 Fetching data for 103 wallets...



  0%|          | 0/103 [00:00<?, ?it/s]

100%|██████████| 103/103 [10:21<00:00,  6.04s/it]


✅ Done. All raw transaction data saved in 'raw_data/' folder.


# Extract & Filter Compound-Specific Interactions

In [54]:
import os
import json
import pandas as pd
from tqdm import tqdm

In [55]:
RAW_DIR = "raw_data"
OUTPUT_CSV = "compound_transactions.csv"


In [56]:
# Heuristic: known Compound V2/V3 method names
COMPOUND_METHODS = [
    "borrow", "repayBorrow", "repayBorrowBehalf",
    "mint", "redeem", "redeemUnderlying",
    "liquidateBorrow"
]


In [57]:
def is_compound_interaction(tx):
    logs = tx.get("log_events", [])
    for log in logs:
        decoded = log.get("decoded", None)
        if decoded:
            method = decoded.get("name", "").lower()
            if any(m.lower() in method for m in COMPOUND_METHODS):
                return True
    return False

In [58]:
def extract_tx_details(tx, wallet_address):
    logs = tx.get("log_events", [])
    method_name = None
    for log in logs:
        decoded = log.get("decoded", None)
        if decoded:
            m = decoded.get("name", "").lower()
            if any(m.lower() in m for m in COMPOUND_METHODS):
                method_name = m
                break

    return {
        "wallet": wallet_address,
        "tx_hash": tx["tx_hash"],
        "block_signed_at": tx["block_signed_at"],
        "method": method_name,
        "from_address": tx["from_address"],
        "to_address": tx["to_address"],
        "value_wei": tx.get("value", "0"),
        "value_usd": tx.get("value_quote", 0.0),
        "gas_price": tx.get("gas_price", 0),
        "gas_offered": tx.get("gas_offered", 0),
        "successful": tx.get("successful", False)
    }

In [59]:
def main():
    all_data = []

    for filename in tqdm(os.listdir(RAW_DIR)):
        if not filename.endswith(".json"):
            continue

        wallet_address = filename.replace(".json", "")
        path = os.path.join(RAW_DIR, filename)

        with open(path, "r") as f:
            content = json.load(f)

        txs = content.get("data", {}).get("items", [])
        for tx in txs:
            if tx.get("log_events") and is_compound_interaction(tx):
                all_data.append(extract_tx_details(tx, wallet_address))

    df = pd.DataFrame(all_data)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"\n✅ Saved filtered Compound transactions to: {OUTPUT_CSV}")


In [60]:
if __name__ == "__main__":
    main()

100%|██████████| 103/103 [00:18<00:00,  5.61it/s]


✅ Saved filtered Compound transactions to: compound_transactions.csv


#  Engineering Risk Features

In [61]:
import pandas as pd
import numpy as np

In [62]:
# Load CSV
df = pd.read_csv('compound_transactions.csv', parse_dates=['block_signed_at'])

In [63]:
# Clean up boolean column
df['successful'] = df['successful'].astype(bool)

In [64]:
# Ensure method column is clean
df['method'] = df['method'].fillna('').str.lower()

In [65]:
# Feature 1: Total Borrowed USD
borrow_df = df[df['method'] == 'borrow']
borrowed = borrow_df.groupby('wallet')['value_usd'].sum().rename('total_borrowed_usd')

In [66]:
# Feature 2: Total Repaid USD
repay_df = df[df['method'].str.contains('repay')]
repaid = repay_df.groupby('wallet')['value_usd'].sum().rename('total_repaid_usd')

In [67]:
# Feature 3: Liquidation Count
liquidated = df[df['method'] == 'liquidateborrow']
liquidation_count = liquidated.groupby('wallet')['tx_hash'].count().rename('liquidation_count')

In [68]:
# Feature 4: Activity Score (first and last tx + duration)
activity = df.groupby('wallet')['block_signed_at'].agg(['min', 'max']).rename(columns={'min': 'first_tx_date', 'max': 'last_tx_date'})
activity['activity_days'] = (activity['last_tx_date'] - activity['first_tx_date']).dt.days + 1

In [69]:
# Feature 5: Bot Behavior
gas_stats = df.groupby('wallet')[['gas_price', 'gas_offered']].mean().rename(columns={
    'gas_price': 'avg_gas_price',
    'gas_offered': 'avg_gas_offered'
})

In [70]:
# Feature 6: Transaction Count
tx_count = df.groupby('wallet')['tx_hash'].count().rename('tx_count')

In [71]:
# Merge All Features
features = (
    borrowed.to_frame()
    .join(repaid, how='outer')
    .join(liquidation_count, how='outer')
    .join(activity, how='outer')
    .join(gas_stats, how='outer')
    .join(tx_count, how='outer')
)

In [72]:
# Fill NaNs with 0 for numerical features
features.fillna({
    'total_borrowed_usd': 0,
    'total_repaid_usd': 0,
    'liquidation_count': 0,
    'avg_gas_price': 0,
    'avg_gas_offered': 0,
    'tx_count': 0,
    'activity_days': 0
}, inplace=True)

In [74]:
# Preview
features.reset_index().head(100)

,wallet,total_borrowed_usd,total_repaid_usd,liquidation_count,first_tx_date,last_tx_date,activity_days,avg_gas_price,avg_gas_offered,tx_count
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0.0,0.0,0.0,2020-09-17 07:31:49+00:00,2023-07-17 18:39:59+00:00,1034,1.097344e+11,7.965465e+05,47
1,0x06b51c6882b27cb05e712185531c1f74996dd988,0.0,0.0,0.0,2020-10-16 00:16:57+00:00,2020-10-16 00:16:57+00:00,1,5.000000e+10,2.477770e+05,1
2,0x0795732aacc448030ef374374eaae57d2965c16c,0.0,0.0,0.0,2020-11-25 13:13:37+00:00,2020-11-25 13:13:37+00:00,1,5.100000e+10,2.477770e+05,1
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,0.0,0.0,0.0,2019-07-28 05:17:20+00:00,2020-12-12 06:15:09+00:00,504,3.770000e+10,6.410347e+05,3
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,0.0,0.0,0.0,2020-10-21 13:02:48+00:00,2020-10-21 13:02:48+00:00,1,4.300000e+10,2.477770e+05,1
...,...,...,...,...,...,...,...,...,...,...
95,0xf10fd8921019615a856c1e95c7cd3632de34edc4,0.0,0.0,0.0,2020-07-05 12:54:20+00:00,2020-09-20 12:38:15+00:00,77,7.950000e+10,3.120200e+05,2
96,0xf340b9f2098f80b86fbc5ede586c319473aa11f3,0.0,0.0,0.0,2021-03-19 20:35:51+00:00,2025-05-06 08:54:59+00:00,1509,1.567909e+11,2.852412e+05,25
97,0xf54f36bca969800fd7d63a68029561309938c09b,0.0,0.0,0.0,2020-12-14 07:49:21+00:00,2020-12-14 07:49:21+00:00,1,4.180000e+10,2.477770e+05,1
98,0xf60304b534f74977e159b2e159e135475c245526,0.0,0.0,0.0,2020-02-06 07:11:32+00:00,2020-03-21 15:53:16+00:00,45,2.531312e+10,1.451494e+06,10


In [75]:
# Save features to CSV
features.reset_index().to_csv('compound_wallet_features.csv', index=False)

# Risk Scoring Model

In [90]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [91]:
def load_features(file_path="compound_wallet_features.csv"):
    return pd.read_csv(file_path)

In [92]:
def normalize_features(df, feature_cols, inverse_cols=[]):
    scaler = MinMaxScaler()
    df_scaled = df.copy()

    for col in feature_cols:
        if col in inverse_cols:
            df_scaled[col] = 1 - scaler.fit_transform(df[[col]])
        else:
            df_scaled[col] = scaler.fit_transform(df[[col]])

    return df_scaled

In [93]:
def compute_risk_score(df):
    # Define weights
    weights = {
        "total_borrowed_usd": 0.15,
        "total_repaid_usd": 0.20,
        "liquidation_count": 0.15,     # inverse
        "activity_days": 0.15,
        "avg_gas_price": 0.10,         # inverse
        "avg_gas_offered": 0.05,       # inverse
        "tx_count": 0.20
    }

    features = list(weights.keys())
    inverse_cols = ["liquidation_count", "avg_gas_price", "avg_gas_offered"]

    df_norm = normalize_features(df, features, inverse_cols)

    # Weighted sum
    df["score"] = df_norm[features].dot([weights[col] for col in features])
    df["score"] = (df["score"] * 1000).round().astype(int)

    return df[["wallet", "score"]]

In [94]:
def save_scores(df, output_path="compound_wallet_risk_scores.csv"):
    df.to_csv(output_path, index=False)
    print(f"✅ Saved risk scores to: {output_path}")

In [95]:
if __name__ == "__main__":
    df_features = load_features("compound_wallet_features.csv")
    df_scores = compute_risk_score(df_features)
    save_scores(df_scores)

✅ Saved risk scores to: compound_wallet_risk_scores.csv
